## Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import requests

# ----------- Science Jubilee -------------
from science_jubilee import Machine as Jub
from science_jubilee.tools import HTTPSyringe as syringe
from science_jubilee.tools import Pipette
from science_jubilee.utils import Handlers
import time
import numpy as np
import pandas as pd
import logging
import sys
sys.path.append('../..')
import stober_synthesis_utils as stober

In [3]:
FORMAT = '%(asctime)s:%(levelname)s:%(name)s:%(message)s'
logging.basicConfig(filename = '2025_02_25_Mixcontamtest_StirPlate.log', level = logging.INFO, format = FORMAT)
logger = logging.getLogger(__name__)

In [4]:
jubilee = Jub.Machine(address='192.168.1.2', simulated = False, crash_detection = False) 

In [5]:
sample_table = pd.read_csv('SampleTable_ReproReplicates_10000.csv')
#sample_table_2 = pd.read_csv('SampleTable_ReproReplicates_10000.csv')


In [6]:
deck = jubilee.load_deck('lab_automation_deck_AFL_bolton.json')

In [8]:
sample_table['well'] = ['A1', 'A2', 'A3', 'A4', 'B1']

In [9]:
sample_table

,Unnamed: 0,uuid,teos_volume,ammonia_volume,water_volume,ethanol_volume,dilution_volume_fraction,silica_mass_conc,silica_mass_fraction,well
0,0,82c085e1-a523-40ce-9e89-4fa0bf546525,214.491,538.3935,1249.9815,7997.134,2.965909,2.025686,0.002527,A1
1,1,747534b1-7370-49a5-bdc1-4fce5227f391,214.491,538.3935,1249.9815,7997.134,2.965909,2.025686,0.002527,A2
2,2,e5f6e82f-0282-46c8-be77-d8914adca9b5,214.491,538.3935,1249.9815,7997.134,2.965909,2.025686,0.002527,A3
3,3,c87ac840-ddde-43ab-a8e3-b5e3eb5a273d,214.491,538.3935,1249.9815,7997.134,2.965909,2.025686,0.002527,A4
4,4,d034d82c-e329-484c-91cc-75ea655a1446,214.491,538.3935,1249.9815,7997.134,2.965909,2.025686,0.002527,B1


## Load Tools

In [10]:
syringe_10 = syringe.HTTPSyringe.from_config(2, "../../../science-jubilee/src/science_jubilee/tools/configs/10cc_syringe.json")
syringe_1_1 = syringe.HTTPSyringe.from_config(1, "../../../science-jubilee/src/science_jubilee/tools/configs/1cc_1_syringe.json")
syringe_1_2 = syringe.HTTPSyringe.from_config(3, "../../../science-jubilee/src/science_jubilee/tools/configs/1cc_2_Hamiltonsyringe.json")
syringe_1_3 = syringe.HTTPSyringe.from_config(4, "../../../science-jubilee/src/science_jubilee/tools/configs/1cc_3_Hamiltonsyringe.json")

Syringe name:  10cc_1
{'capacity': 10000, 'empty_position': 1830, 'full_position': 1217, 'name': '10cc_1'}
Syringe name:  1cc_1
{'capacity': 1000, 'empty_position': 1845, 'full_position': 1240, 'name': '1cc_1'}
Syringe name:  1cc_2_hamilton
{'capacity': 1000, 'empty_position': 1753, 'full_position': 1120, 'name': '1cc_2_hamilton'}
Syringe name:  1cc_3_hamilton
{'capacity': 1000, 'empty_position': 1753, 'full_position': 1120, 'name': '1cc_3_hamilton'}


In [11]:
jubilee.load_tool(syringe_10)
jubilee.load_tool(syringe_1_1)
jubilee.load_tool(syringe_1_2)
jubilee.load_tool(syringe_1_3)

In [12]:
mix_syringe = syringe_10
water_syringe = syringe_1_1
ammonia_syringe = syringe_1_2
teos_syringe = syringe_1_3

## Load water syringe with water

In [13]:
water_syringe.load_syringe(600, 1500)

Loaded syringe, remaining volume 600 uL


In [20]:
water_syringe.set_pulsewidth(water_syringe.empty_position-1, s = 2000)

In [21]:
water_syringe.set_pulsewidth(water_syringe.full_position+1, s = 150)

In [22]:
water_syringe.set_pulsewidth(water_syringe.full_position+155, s = 500)

In [23]:
water_syringe.load_syringe(740, water_syringe.full_position+155)

Loaded syringe, remaining volume 740 uL


## Load ammonia syringe

In [24]:
#ammonia_syringe.set_pulsewidth(1420, s = 10)

In [25]:
ammonia_syringe.load_syringe(600, 1500)

Loaded syringe, remaining volume 600 uL


In [32]:
ammonia_syringe.set_pulsewidth(ammonia_syringe.empty_position-1, s = 2000)

In [33]:
ammonia_syringe.set_pulsewidth(ammonia_syringe.full_position+1, s = 200)

In [34]:
ammonia_syringe.set_pulsewidth(ammonia_syringe.full_position+250, s = 500)

In [35]:
ammonia_syringe.load_syringe(560, ammonia_syringe.full_position+250)

Loaded syringe, remaining volume 560 uL


## Load TEOS syringe

In [36]:
teos_syringe.load_syringe(600, 1500)

Loaded syringe, remaining volume 600 uL


In [41]:
teos_syringe.set_pulsewidth(teos_syringe.empty_position-1, s = 2000)

In [42]:
teos_syringe.set_pulsewidth(teos_syringe.full_position+1, s = 200)

In [43]:
teos_syringe.set_pulsewidth(teos_syringe.full_position+300, s = 200)

In [44]:
teos_syringe.load_syringe(480, teos_syringe.full_position+300)

Loaded syringe, remaining volume 480 uL


## load ethanol syringe

In [45]:
#jubilee.park_tool()

In [49]:
mix_syringe.load_syringe(0, mix_syringe.empty_position-1)

Loaded syringe, remaining volume 0 uL


In [50]:
mix_syringe.set_pulsewidth(mix_syringe.empty_position-1, s = 2000)


In [51]:
mix_syringe.set_pulsewidth(mix_syringe.full_position+1, s = 1000)

In [52]:
#mix_syringe.empty_position

In [53]:
mix_syringe.set_pulsewidth(mix_syringe.full_position+250, s = 500)

In [54]:
mix_syringe.load_syringe(6200, mix_syringe.full_position+250)

Loaded syringe, remaining volume 6200 uL


## Load labware


In [55]:
samples = jubilee.load_labware('20mlscintillation_12_wellplate_18000ul.json', 2)
samples.manual_offset([(31.1, 169.0), (121.1, 169.0), (121.1, 113.0)])

New manual offset applied to 20mlscintillation_12_wellplate_18000ul


In [56]:
stocks_main = jubilee.load_labware('20mlscintillation_12_wellplate_18000ul.json', 4)
stocks_main.manual_offset([(30.5, 266.0),  (120.5, 266.0), (120.5, 210.0)])

New manual offset applied to 20mlscintillation_12_wellplate_18000ul


In [57]:
trash = jubilee.load_labware('agilent_1_reservoir_290ml.json', 1)

In [59]:
teos_stocks = [stocks_main[0]]
ammonia_stocks = [stocks_main[1]]
water_stocks = [stocks_main[2]]
ethanol_stocks_apdist = [stocks_main[3], stocks_main[4], stocks_main[5], stocks_main[6]]
rinse_stocks_apdist = [stocks_main[9], stocks_main[10], stocks_main[11]]

## Check sample alignment

In [47]:
jubilee.pickup_tool(mix_syringe)
for i, row in sample_table.iterrows():
    loc = samples[row['well']]
    jubilee.move_to(x = loc.x, y = loc.y)
    print(loc.y)
    jubilee.move_to(z = loc.top_+7)
    curr_well = row['well']
    ans = input(f'Is the tip centered over sample {curr_well}?')
    if ans == 'y':
        continue
    else:
        continue
    

169.0


Is the tip centered over sample A1? 


169.0


Is the tip centered over sample A2? 


169.0


...

Is the tip centered over sample B4? 


113.0


Is the tip centered over sample C1? 


113.0


Is the tip centered over sample C2? 


In [57]:
sample_table

,Unnamed: 0,uuid,teos_volume,ammonia_volume,water_volume,ethanol_volume,dilution_volume_fraction,silica_mass_conc,silica_mass_fraction,well
0,0,82c085e1-a523-40ce-9e89-4fa0bf546525,214.491,538.3935,1249.9815,7997.134,2.965909,2.025686,0.002527,A1
1,1,747534b1-7370-49a5-bdc1-4fce5227f391,214.491,538.3935,1249.9815,7997.134,2.965909,2.025686,0.002527,A2
2,2,e5f6e82f-0282-46c8-be77-d8914adca9b5,214.491,538.3935,1249.9815,7997.134,2.965909,2.025686,0.002527,A3
3,3,c87ac840-ddde-43ab-a8e3-b5e3eb5a273d,214.491,538.3935,1249.9815,7997.134,2.965909,2.025686,0.002527,A4
4,4,d034d82c-e329-484c-91cc-75ea655a1446,214.491,538.3935,1249.9815,7997.134,2.965909,2.025686,0.002527,B1


In [57]:
jubilee.park_tool()


In [106]:
loc = samples['A8']
jubilee.move_to(x = loc.x, y = loc.y)
print(loc.y)
jubilee.move_to(z = loc.top_+7)

175.8614659806774


# Experiment


In [60]:
location_lookup = {row['uuid']:samples[row['well']] for i, row in sample_table.iterrows()}


In [61]:
sample_table['ethanol_volume'].sum()

39985.67

In [62]:
sample_table.iloc[-1:]

,Unnamed: 0,uuid,teos_volume,ammonia_volume,water_volume,ethanol_volume,dilution_volume_fraction,silica_mass_conc,silica_mass_fraction,well
4,4,d034d82c-e329-484c-91cc-75ea655a1446,214.491,538.3935,1249.9815,7997.134,2.965909,2.025686,0.002527,B1


In [63]:
ethanol_stocks_apdist[-1:]

[Well B3 form 20mLscintillation 12 Well Plate 18000 µL on slot 4]

## add ethanol

In [64]:
stober.add_reactants_batch(jubilee, mix_syringe, mix_syringe, sample_table, location_lookup, 'ethanol_volume', ethanol_stocks_apdist, stocks_usable_volume=15000, dwell_time = 10)

In [59]:
#dispense remaining volume from etOH mix syringe
jubilee.pickup_tool(mix_syringe)
mix_syringe.dispense(mix_syringe.remaining_volume-1, trash[0])
logger.info('Emptied ethanol syringe into trash')
jubilee.park_tool()

In [60]:
mix_syringe.set_pulsewidth(mix_syringe.empty_position-1)

## Dispense water


In [65]:
stober.add_reactants_batch(jubilee, water_syringe, mix_syringe, sample_table, location_lookup, 'water_volume', water_stocks, dwell_time = 7)

## dispense Ammonia

In [66]:
stober.add_reactants_batch(jubilee, ammonia_syringe, mix_syringe, sample_table, location_lookup, 'ammonia_volume', ammonia_stocks)

## mix precursors

In [63]:
stober.first_mix(jubilee, mix_syringe, 5000, location_lookup, rinse_stocks_apdist, 5)

## Add TEOS and mix

In [68]:
stober.add_reactants_batch(jubilee, teos_syringe, mix_syringe, sample_table.iloc[1:], location_lookup, 'teos_volume', teos_stocks, wait = True)

Hit any key when ready to continue 
Hit any key when ready to continue 
Hit any key when ready to continue 
Hit any key when ready to continue 
